# Gorilla - Python assessment

## Task 1: Use pandas to calculate a transportation distribution charge for four gas meters in the United Kingdom

In [1]:
# import libraries
import pandas as pd
import numpy as np
import time

In [24]:
# Input data
file_path = "./gorilla_test_data.xlsx"
meter_list = pd.read_excel(file_path, sheet_name="meter_list")
meter_list.index = meter_list["meter_id"]
forecast_table = pd.read_excel(file_path, sheet_name="forecast_table")
rate_table = pd.read_excel(file_path, sheet_name="rate_table")


In [25]:
# Calculate total consumption per meter
total_cons_p_meter = forecast_table.drop(columns=["date"]).groupby("meter_id").sum()
total_cons_p_meter

,kwh
meter_id,
14676236,28978.0
34509937,78324.0
50264822,265667.0
88357331,484399.0


In [27]:
# Calculate total costs per exit zone
total_cons_p_meter = forecast_table.drop(columns=["date"]).groupby("meter_id").sum()
rate_per_e_z = pd.concat([meter_list, total_cons_p_meter], join = "outer", axis = 1)
rate_per_e_z = rate_per_e_z.merge(rate_table, on = "exit_zone", how = "inner")
rate_per_e_z["aq_max_kwh"] = rate_per_e_z["aq_max_kwh"].fillna(float("inf"))
rate_per_e_z = rate_per_e_z[(rate_per_e_z["aq_min_kwh"] <= rate_per_e_z["kwh"]) & 
                            (rate_per_e_z["kwh"] < rate_per_e_z["aq_max_kwh"])]

# Add right rates to right dates
rate_per_e_z["end_date"] = rate_per_e_z.groupby("meter_id").date.shift(-1) - pd.Timedelta(days=1) 
forecast_rate = pd.merge(forecast_table, rate_per_e_z, on = ["meter_id"])
forecast_rate["end_date"] = forecast_rate["end_date"].fillna(pd.Timestamp.max)
forecast_rate = forecast_rate[(forecast_rate["date_x"] <= forecast_rate["end_date"]) & 
                              (forecast_rate["date_x"] >= forecast_rate["date_y"])]

# Calculate costs in GBP and sum per meter
forecast_rate["cost_per_kwh"] = forecast_rate["kwh_x"] * forecast_rate["rate_p_per_kwh"] * 0.01
forecast_rate = forecast_rate[["meter_id", "kwh_x", "cost_per_kwh"]].groupby("meter_id").sum()

# Output data
forecast_rate = forecast_rate.rename(columns={"kwh_x": "Total_estimated_Consumption_kWh", 
                                              "cost_per_kwh": "Total_cost_gbp"})
forecast_rate["meter_id"] = forecast_rate.index
forecast_rate = forecast_rate.reset_index(drop=True)
forecast_rate.round(2)

,Total_estimated_Consumption_kWh,Total_cost_gbp,meter_id
0,28978.0,100.15,14676236
1,78324.0,275.49,34509937
2,265667.0,731.24,50264822
3,484399.0,1433.16,88357331


## Task 2: Write a function that generates a list of random meters of any size. Examples of valid exit zones can be found in the rate table. You may randomly generate the annual quantity.

In [5]:
def generate_random_meters(size, valid_exit_zones):
    """ Generate a list of random meters
    
    Parameters:
    -----------
    @param size: number of meters to generate
    @type size: int
    @param valid_exit_zones: list of valid exit zones
    @type valid_exit_zones: list
    """
    meter_list = pd.DataFrame({
        "meter_id": np.random.randint(low = 0, high = 999999999, size = size),
        "exit_zone": np.random.choice(valid_exit_zones, size),
        "aq_kwh": np.random.uniform(low = 1, high = 999999999, size = size)
    })
    meter_list.index = meter_list["meter_id"]
    return meter_list
random_meter_list = generate_random_meters(10, np.unique(rate_table["exit_zone"]))
random_meter_list

,meter_id,exit_zone,aq_kwh
meter_id,,,
691523487,691523487,LO,2.028873e+08
725929068,725929068,WA1,6.375576e+04
337276961,337276961,LS,4.422622e+08
275274305,275274305,WA1,9.291276e+08
649727915,649727915,LC,5.706677e+08
48093417,48093417,NO1,4.814795e+08
901073689,901073689,EA1,9.787639e+08
864525207,864525207,SE2,3.817014e+08
453299873,453299873,WA1,6.117792e+07


## Task 3: Write a function that generates mock consumption data given a list of meters and a start date and duration (number of days in the forecast).

In [6]:
def generate_mock_consumption_data(meter_list, start_date, duration):
    """ Generate mock consumption data
    
    Parameters: 
    -----------
    @param meter_list: list of meters
    @type meter_list: pandas.DataFrame
    @param start_date: start date
    @type start_date: datetime string
    @param duration: duration in days
    #type duration: int
    """
    result = pd.DataFrame({
        "meter_id": np.repeat(meter_list["meter_id"], duration),
        "date": np.tile(pd.date_range(start_date, periods = duration), len(meter_list)),
        "kwh": np.random.uniform(low = 1, high = 999999999, size = len(meter_list) * duration)
    })
    return(result.reset_index(drop=True))
generate_mock_consumption_data(random_meter_list, "2020-06-01", 2)

,meter_id,date,kwh
0,691523487,2020-06-01,1.903913e+08
1,691523487,2020-06-02,6.429835e+08
2,725929068,2020-06-01,7.842609e+08
3,725929068,2020-06-02,1.165693e+08
4,337276961,2020-06-01,5.568875e+08
5,337276961,2020-06-02,1.869889e+07
6,275274305,2020-06-01,2.552217e+08
7,275274305,2020-06-02,4.609445e+08
8,649727915,2020-06-01,1.824165e+08
9,649727915,2020-06-02,2.494019e+08


## Task 4
### A: Write a function that takes as an input a meter list and a consumption forecast table and that calculates the transportation cost table

In [28]:
def calculate_T_cost_table(meter_list, forecast_table, rate_table):
    """Calculate the transportation cost table
    
    Parameters:
    -----------
    @param meter_list: list of meters
    @type meter_list: pandas.DataFrame
    @param forecast_table: forecast table
    @type forecast_table: pandas.DataFrame
    @param rate_table: rate table
    @type rate_table: pandas.DataFrame
    """
    # Calculate total costs per exit zone
    total_cons_p_meter = forecast_table.drop(columns=["date"]).groupby("meter_id").sum()
    rate_per_e_z = pd.concat([meter_list, total_cons_p_meter], join = "outer", axis = 1)
    rate_per_e_z = rate_per_e_z.merge(rate_table, on = "exit_zone", how = "inner")
    rate_per_e_z["aq_max_kwh"] = rate_per_e_z["aq_max_kwh"].fillna(float("inf"))
    rate_per_e_z = rate_per_e_z[(rate_per_e_z["aq_min_kwh"] <= rate_per_e_z["kwh"]) & 
                                (rate_per_e_z["kwh"] < rate_per_e_z["aq_max_kwh"])]

    # Add right rates to right dates
    rate_per_e_z["end_date"] = rate_per_e_z.groupby("meter_id").date.shift(-1) - pd.Timedelta(days=1) 
    forecast_rate = pd.merge(forecast_table, rate_per_e_z, on = ["meter_id"])
    forecast_rate["end_date"] = forecast_rate["end_date"].fillna(pd.Timestamp.max)
    forecast_rate = forecast_rate[(forecast_rate["date_x"] <= forecast_rate["end_date"]) & 
                                (forecast_rate["date_x"] >= forecast_rate["date_y"])]

    # Calculate costs in GBP and sum per meter
    forecast_rate["cost_per_kwh"] = forecast_rate["kwh_x"] * forecast_rate["rate_p_per_kwh"] * 0.01
    forecast_rate = forecast_rate[["meter_id", "kwh_x", "cost_per_kwh"]].groupby("meter_id").sum()

    # Output data
    forecast_rate = forecast_rate.rename(columns={"kwh_x": "Total_estimated_Consumption_kWh", 
                                                "cost_per_kwh": "Total_cost_gbp"})
    forecast_rate["meter_id"] = forecast_rate.index
    forecast_rate = forecast_rate.reset_index(drop=True)
    return forecast_rate.round(2)

In [29]:
random_meter_list = generate_random_meters(10, np.unique(rate_table["exit_zone"]))
calculate_T_cost_table(meter_list = random_meter_list, 
                       forecast_table = generate_mock_consumption_data(random_meter_list, "2020-06-01", 2),
                       rate_table=rate_table)


,Total_estimated_Consumption_kWh,Total_cost_gbp,meter_id
0,8.473186e+08,2454681.95,133587612
1,1.152661e+09,3313899.50,293053103
2,1.352986e+09,3371641.55,525780209
3,7.764777e+08,1758722.07,572840068
4,8.401716e+08,1553477.28,632788112
5,8.697061e+08,2410825.37,654239802
6,1.695535e+09,3426677.12,814166829
7,6.954592e+08,1235135.47,887937739
8,1.609238e+09,3611130.62,908707614
9,1.498172e+09,4152932.43,930967732


### B: Benchmark this function using meter lists of different sizes and consumption forecasts for periods of different lengths

In [32]:
parameters = pd.DataFrame({
    "meter_size": [10, 50, 100, 1000, 10000, 10, 10, 10,  10, 10, 50, 100, 1000, 5000],
    "duration": [10, 10, 10, 10, 10, 50, 100, 1000, 10000, 50000, 50, 100, 1000, 5000],
})
results = np.empty((0,3))
for i, row in parameters.iterrows():
    meter_list = generate_random_meters(row["meter_size"], np.unique(rate_table["exit_zone"]))
    forecast_table = generate_mock_consumption_data(meter_list, "2020-06-01", row["duration"])
    start = time.time()
    try:
        calculate_T_cost_table(meter_list = meter_list, 
                       forecast_table = forecast_table,
                       rate_table=rate_table)
        end = time.time()
        total_time = end - start
    except:
        total_time = "error"
    results = np.vstack([results, [row["meter_size"],  row["duration"], total_time]])
pd.DataFrame(results, columns = ["meter_size", "duration", "total_time"])


,meter_size,duration,total_time
0,10.0,10.0,0.018995
1,50.0,10.0,0.015997
2,100.0,10.0,0.013977
3,1000.0,10.0,0.122929
4,10000.0,10.0,0.224681
5,10.0,50.0,0.010032
6,10.0,100.0,0.010998
7,10.0,1000.0,0.028967
8,10.0,10000.0,0.136825
9,10.0,50000.0,0.772531


### C: How does the function scale for larger sets of data?


This function scales up exponentially for both the meter size and the number of days in the forecast. Once we have both a large number of meter_size and number of days, the function runs very slowly.


## Task 5: What are your observations after benchmarking? Are there any steps in the cost calculation that can be improved? How would you go about improving the performance of this calculation?

The code is slow for larger datasets. We could rewrite the code with for loops and run it in parallel using numba. This will precompile the code to machine language which greatly improves speed withoutn needing to learn C or C++ (or any other fast language). Running the loops in parallel will also largely increase the speed. To my knowledge the resulting dataframes will also become increasingly large, so doing everything sequentially instead of vectorized might solve the memory issues.